In [ ]:
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK

In [ ]:
!bunzip2 shape_predictor_68_face_landmarks.dat.bz2

datFile =  "shape_predictor_68_face_landmarks.dat"

In [ ]:
!pip install imutils
!pip3 install face_recognition

In [1]:
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
import uuid 
import os
import math
import glob 
import face_recognition
from collections import deque
# cap = cv2.VideoCapture(0)

In [2]:
def take_known_picture():
    cap = cv2.VideoCapture(0)
    name = str(input("enter name of the picture: "))
    known_dir="known_images/"+name
    # ret,frame = cap.read()
    # cv2.imshow("name",frame)
    if not (os.path.isdir(known_dir)):
        mode = 0o777
        os.makedirs(known_dir,mode)
    while(cap.isOpened()):
        ret,frame = cap.read()
        cv2.imshow("name",frame) #display the captured image
        if cv2.waitKey(1) & 0xFF == ord('y'): #save on pressing 'y' 
            file_path = known_dir+'/01.jpg'
            cv2.imwrite(file_path,frame)
            cv2.destroyAllWindows()
            break
    return name

In [34]:
name = take_known_picture()

[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (935) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


enter name of the picture:  nafees


In [2]:
def midpoint(point1 ,point2):
    return (point1.x + point2.x)/2,(point1.y + point2.y)/2

def euclidean_distance(point1 , point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def get_blink_ratio(eye_points, facial_landmarks):
    
    #loading all the required points
    corner_left  = (facial_landmarks.part(eye_points[0]).x, 
                    facial_landmarks.part(eye_points[0]).y)
    corner_right = (facial_landmarks.part(eye_points[3]).x, 
                    facial_landmarks.part(eye_points[3]).y)
    
    center_top    = midpoint(facial_landmarks.part(eye_points[1]), 
                             facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), 
                             facial_landmarks.part(eye_points[4]))

    #calculating distance
    horizontal_length = euclidean_distance(corner_left,corner_right)
    vertical_length = euclidean_distance(center_top,center_bottom)

    ratio = horizontal_length / vertical_length
    # print("ratio: " + str(ratio))
    return ratio

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
# cap = cv2.VideoCapture(0)
left_eye_landmarks  = [36, 37, 38, 39, 40, 41]
right_eye_landmarks = [42, 43, 44, 45, 46, 47]
def blinking_count(expected_blinks):
    blinks = 0
    BLINK_RATIO_THRESHOLD = 5.2
    end_time = time.time()+20
    result = False
    time_queue = deque([0]*3,maxlen=3)
    cap = cv2.VideoCapture(0)
    while (cap.isOpened()):
        # Capture the image from the webcam
        ret, image = cap.read()

        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break 

        # Convert the image color to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Detect the face
        # rects = detector(gray, 1)
        # Detect landmarks for each face
        faces,_,_ = detector.run(image = image, upsample_num_times = 0, 
                           adjust_threshold = 0.0)
        rects=faces
        for rect in rects:
            # Get the landmark points
            shape = predictor(gray, rect)
            left_eye_ratio  = get_blink_ratio(left_eye_landmarks, shape)
            right_eye_ratio = get_blink_ratio(right_eye_landmarks, shape)
            blink_ratio     = (left_eye_ratio + right_eye_ratio) / 2
        # Convert it to the NumPy Array
            shape_np = np.zeros((68, 2), dtype="int")

            for i in range(0, 68):
                shape_np[i] = (shape.part(i).x, shape.part(i).y)
            shape = shape_np

            # Display the landmarks
            for i, (x, y) in enumerate(shape):
            # Draw the circle to mark the keypoint 
                cv2.circle(image, (x, y), 1, (0, 0, 255), -1)

            # text = "BLINK_COUNT: "+str(blinks)
            if blink_ratio > BLINK_RATIO_THRESHOLD:
                #Blink detected! Do Something!

                blinks = blinks+1
                time.sleep(.125)
                time_queue.append(round(time.time()*1000))
                if not (time_queue[0]== 0 or time_queue[2] == 0):
                    if (time_queue[2]-time_queue[0]<1000):
                        # print("bot_time :"+str(time_queue[2]-time_queue[0]))
                        blinks = 0
            text = "BLINK_COUNT: "+str(blinks)
            cv2.putText(image,text,(10,50), cv2.FONT_HERSHEY_SIMPLEX,
                        2,(0,0,0),2,cv2.LINE_AA)
            cv2.putText(image,str(blink_ratio),(60,100), cv2.FONT_HERSHEY_SIMPLEX,
                            2,(0,0,0),2,cv2.LINE_AA)
            if (blinks >=expected_blinks):
                result = True
                cv2.destroyAllWindows()
                return result

        # Display the image
        cv2.imshow('BlinkDetector', image)

        if (time.time() > end_time):
            # print("please follow the instructions")
            cv2.destroyAllWindows()
            break
        # Press the escape button to terminate the code
        if cv2.waitKey(10) == 27:
            cv2.destroyAllWindows()
            break

    cap.release()
    return result

In [4]:
my_list = os.listdir('known_images')
my_list
known_face_names=[]
known_face_encodings=[]
for i in range(len(my_list)):
    if(my_list[i]!=".ipynb_checkpoints"):

        image=face_recognition.load_image_file("known_images/"+my_list[i]+"/01.jpg")
        face_encoding = face_recognition.face_encodings(image)[0]
        known_face_encodings.append(face_encoding)
        known_face_names.append(my_list[i])

In [5]:
def face_recognize():
    video_capture = cv2.VideoCapture(0)
    while (video_capture.isOpened()):
        # Grab a single frame of video
        ret, frame = video_capture.read()

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Find all the faces and face enqcodings in the frame of video
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Loop through each face in this frame of video
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

            name = "Unknown"

            # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(10) == 27:
            cv2.destroyAllWindows()
            break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()

In [6]:
blink_num=int(input("how many blinks?"))
res = blinking_count(blink_num)
if res:
    face_recognize()
else:
    print("please follow the instructions")


how many blinks? 4


[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (935) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (935) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
